In [24]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import pipeline

In [19]:
d1= pd.read_csv(r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v1_intents\processed_with_intents_refined_pet.tsv", sep='\t')
d2= pd.read_csv(r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v1_intents\processed_with_intents_refined_health.tsv", sep='\t')
d3= pd.read_csv(r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v1_intents\processed_with_intents_refined_beauty.tsv", sep='\t')
d4= pd.read_csv(r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\processed_expanded_stratified_reviews_health_v2_with_intents.tsv", sep='\t')
d5= pd.read_csv(r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\processed_expanded_stratified_reviews_beauty_v2_with_intents.tsv", sep='\t')
d6= pd.read_csv(r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\processed_expanded_stratified_reviews_pet_v2_with_intents.tsv", sep='\t')

df = pd.concat([d4,d5,d6], axis=0)
df.review_combined = df.review_combined.str.replace(':',' ')
df.review_combined = df.review_combined.str.replace('(','')
df.review_combined = df.review_combined.str.replace(')','')
df.review_combined = df.review_combined.str.replace('...',' ')
df.review_combined = df.review_combined.str.replace('\"','')
df.review_combined = df.review_combined.str.replace('\'','')
df.review_combined = df.review_combined.str.replace('#','')
df.review_combined = df.review_combined.str.replace('\\','')

df = df[~df.intent.isin(['alternatives','ask_safety','availability'])]

In [40]:
gen_sent  = [
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_side_effect_sentences.tsv",
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_instructions_sentences.tsv",
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_features_sentences.tsv",
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_sentences_features_intent.tsv",
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_sentences_ask_shipping_intent.tsv",
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_sentences_ask_usage_intent.tsv",
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_sentences_compare_products_intent.tsv",
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_sentences_ask_features_intent.tsv",
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_sentences_instructions_intent.tsv",
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_sentences_report_side_effect_intent.tsv",
r"C:\Users\91962\Downloads\Course Material\CS9873-BrainInspiredAI\v2_intents\generated_sentences_return_and_refund_intent.tsv"
]

cols = ['report_side_effect','instructions','features', 'features', 'ask_shipping', 'ask_usage', 'compare_products', 'ask_features', 'instructions', 
'report_side_effect', 'return_and_refund']

for idx, gen in enumerate(gen_sent):
    tmp  = pd.read_csv(gen, sep='\t')
    tmp['intent'] = cols[idx]
    tmp.rename(columns={tmp.columns[0]:'review_combined'}, inplace=True)

    df = pd.concat([df,tmp], axis=0)

In [41]:
df[['intent', 'review_combined']].groupby(by= 'intent').count()

,review_combined
intent,
ask_features,848
ask_packaging,1692
ask_quantity,1314
ask_result,799
ask_shipping,837
ask_usage,864
compare_products,889
express_dissatisfaction,15037
express_satisfaction,33487


In [25]:
pipe = pipeline("text2text-generation", model="EngLip/flan-t5-sentence-generator")

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [39]:
pipe("meaning usually i would eat big portions of food twice a day.")

[{'generated_text': "I'm not a big eater."}]

In [38]:
df.review_combined.values[74]

'meaning usually i would eat big portions of food twice a day.'

In [113]:
df['garbage'] = 1

In [114]:
X = df.drop(columns='garbage')
y = df.garbage

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 28, stratify= X.intent)

In [116]:
df.groupby(by= 'intent').count()

,customer_id,review_id,product_id,product_title,product_category,star_rating,helpful_votes,vine,verified_purchase,review_combined,sentiment_label,sentiment_score,garbage
intent,,,,,,,,,,,,,
ask_features,448,448,448,448,448,448,448,448,448,448,448,448,448
ask_packaging,1692,1692,1692,1692,1692,1692,1692,1692,1692,1692,1692,1692,1692
ask_quantity,1314,1314,1314,1314,1314,1314,1314,1314,1314,1314,1314,1314,1314
ask_result,799,799,799,799,799,799,799,799,799,799,799,799,799
ask_shipping,437,437,437,437,437,437,437,437,437,437,437,437,437
ask_usage,464,464,464,464,464,464,464,464,464,464,464,464,464
compare_products,589,589,589,589,589,589,589,589,589,589,589,589,589
express_dissatisfaction,15037,15037,15037,15037,15037,15037,15037,15037,15037,15037,15037,15037,15037
express_satisfaction,33487,33487,33487,33487,33487,33487,33487,33487,33487,33487,33487,33487,33487


In [117]:
df[df.intent == 'report_side_effect']

,customer_id,review_id,product_id,product_title,product_category,star_rating,helpful_votes,vine,verified_purchase,review_combined,sentiment_label,sentiment_score,intent,garbage
307,39481955.0,rb7se7j50zfch,b0087spcxw,"tls acts adrenal, cartisol, thyroid & stress s...",health & personal care,5.0,5.0,n,y,i have been taking it for approximately six we...,neutral,0.630644,report_side_effect,1
641,32311711.0,r1rriyut0lvrq5,b003i3sueo,authentic black ant natural sexual enhancer fo...,health & personal care,4.0,1.0,n,y,the best thing about this product is that ther...,positive,0.733761,report_side_effect,1
707,25354838.0,ryrozupr77hsm,b005lqx5q4,sensa months 3 & 4 60 day weight-loss system -...,health & personal care,1.0,0.0,n,n,i tried it one time and had an allergic reacti...,negative,0.828942,report_side_effect,1
1106,36861245.0,rskt8x5kbqxik,b00pe5wbrg,horny goat weed w/ maca root & 20% icariins | ...,health & personal care,2.0,0.0,n,n,"not feeling it, looks good, sounds hopeful but...",positive,0.461373,report_side_effect,1
1327,51991045.0,rinpffpp4bqhp,b005gplt9a,jovial first ever einkorn pasta linguine -- 12 oz,health & personal care,5.0,4.0,n,y,people with gluten sensitivity reading this ma...,neutral,0.389506,report_side_effect,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Unlike other products, the dietary supplement ...",NaN,NaN,report_side_effect,1
296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"After a week of using this cream, an allergic ...",NaN,NaN,report_side_effect,1
297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I found the cat food effective, but I wish the...",NaN,NaN,report_side_effect,1
298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I found the dietary supplement effective, but ...",NaN,NaN,report_side_effect,1


In [42]:
def write_yml_segment(grouped):
    intent = grouped.intent.unique()[0]
    grouped = grouped.sample(799, random_state=100)
    print(intent, len(grouped))
    text = f"""- intent: {intent}
  examples: |\n"""
    for review in grouped.review_combined.tolist():
        text += f"    - {review}\n"
    return text

In [43]:
segments = df.groupby(by= 'intent').apply(write_yml_segment)

ask_features 799
ask_packaging 799
ask_quantity 799
ask_result 799
ask_shipping 799
ask_usage 799
compare_products 799
express_dissatisfaction 799
express_satisfaction 799
features 799
instructions 799
pricing 799
recommendation 799
report_side_effect 799
return_and_refund 799
usage 799
verify_authenticity 799


In [44]:
text = "nlu:\n"
for seg in segments:
    text += seg
    text += "\n"

In [45]:
text

"nlu:\n- intent: ask_features\n  examples: |\n    - the soap i purchased in las vegas was manufactured in the u.s. i would think the ingredients would be the same, but the soap manufactured in switzerland just doesnt have the same wow factor that the u.s. manufactured soap has.\n    - i have to crush the vitamin and put it in peanut butter in order to get him to eat it.\n    - What features set this product apart from its competitors?\n    - since it can be hard to get enough vitamin d in one day, i got these and was pleasantly surprised with the size of the pills.\n    - Can you provide more details about the critical features of this product?\n    - this is the most efficient and hassle free way to ingest these ingredients that i have encountered.\n    - What features set this product apart from its competitors?\n    - What features set this product apart from its competitors?\n    - i feel all right with the size of the pill, but it does look relatively big, although it is no bigger

In [47]:
with open('./train_nlu.yml', 'w+', encoding="utf-8") as f:
    f.write(text)

In [50]:
def clean_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()

    # Remove the SUB character and any other control characters except newline and tab
    clean_content = ''.join(char for char in content if char.isprintable() or char in '\n\t')

    with open(file_path, "w", encoding="utf-8") as file:
        file.write(clean_content)

file_path = "./train_nlu.yml"  # Adjust the path to your file
clean_file(file_path)


In [49]:
text

"nlu:\n- intent: ask_features\n  examples: |\n    - the soap i purchased in las vegas was manufactured in the u.s. i would think the ingredients would be the same, but the soap manufactured in switzerland just doesnt have the same wow factor that the u.s. manufactured soap has.\n    - i have to crush the vitamin and put it in peanut butter in order to get him to eat it.\n    - What features set this product apart from its competitors?\n    - since it can be hard to get enough vitamin d in one day, i got these and was pleasantly surprised with the size of the pills.\n    - Can you provide more details about the critical features of this product?\n    - this is the most efficient and hassle free way to ingest these ingredients that i have encountered.\n    - What features set this product apart from its competitors?\n    - What features set this product apart from its competitors?\n    - i feel all right with the size of the pill, but it does look relatively big, although it is no bigger

In [151]:
segments_test = df.groupby(by= 'intent').apply(write_yml_segment)

text = "nlu:\n"
for seg in segments_test:
    text += seg
    text += "\n"

with open('./rasa/test_nlu.yml', 'w+', encoding="utf-8") as f:
    f.write(text)

file_path = "./rasa/test_nlu.yml"  # Adjust the path to your file
clean_file(file_path)

ask_features 100
ask_packaging 100
ask_quantity 100
ask_result 100
ask_shipping 100
ask_usage 100
compare_products 100
express_dissatisfaction 100
express_satisfaction 100
features 100
instructions 100
pricing 100
recommendation 100
report_side_effect 100
return_and_refund 100
usage 100
verify_authenticity 100
